In [1]:
import requests
from langchain.schema import Document
import json
from tqdm import tqdm as tqdm
import concurrent.futures
import time
import weaviate
from weaviate.classes.init import Auth
import os
from dotenv import load_dotenv
from parse_jira_tickets import parse_jira_tickets, retry_fetch_ticket, write_to_file

In [3]:
weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_api_key = os.getenv("WEAVIATE_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")  # Or any other inference API keys
    },
    skip_init_checks=True
)

jiratest = client.collections.get("JiraTicket")

In [16]:
start_ticket = 1
num_tickets = 45000
batch_size = 10
max_retries = 5

total_num = len(failed_tickets)

with concurrent.futures.ThreadPoolExecutor(max_workers=40) as executor:  # Adjust max_workers as needed
    future_to_ticket = {executor.submit(retry_fetch_ticket, ticket, email, api_token, max_retries): ticket for ticket in failed_tickets}
    
    results_list = []
    failed_tickets = []
    for future in tqdm(concurrent.futures.as_completed(future_to_ticket), total=total_num):
        try:
            result = future.result()
            write_to_file(result)  # Write the individual result to a file
        except Exception as exc:
            failed_tickets.append(future_to_ticket[future])

100%|██████████| 7763/7763 [1:32:48<00:00,  1.39it/s]  


In [ ]:
final_docs = []
# Push documents to Weaviate
for results in results_list:
    final_docs.append(
        Document(
            page_content=json.dumps(results), 
            metadata={
                "source": f"https://rubinobs.atlassian.net/rest/api/latest/issue/DM-{results['TicketNumber']}",
                "source_key":"jira"
            }
        )
    )

In [7]:
# save failed_tickets
with open('failed_tickets.txt', 'w') as f:
    for ticket in failed_tickets:
        f.write(f"{ticket}\n")

NameError: name 'failed_tickets' is not defined

In [10]:
# Load failed_tickets.txt into a list
with open('failed_tickets.txt', 'r') as f:
    failed_tickets = [int(line.strip()) for line in f.readlines()]

# Now you have a list of failed tickets
print(failed_tickets)

[]


In [12]:
# Define the folder path
folder_path = 'JIRA tickets'

# Check how many files are in the folder
num_files = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
print(f"Number of files: {num_files}")

Number of files: 37239
